In [0]:
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd
import librosa
import librosa.display
import os
from os import path
import io
!pip3 install soundfile
!pip3 install librosa
import soundfile as sf
import librosa.feature


In [0]:
!pip install ipython-autotime

%load_ext autotime

# Opening datasets

In [0]:
DIR = '/content/drive/My Drive/DCASE 2020 Challenge/TAU Urban Acoustic Scenes 2020 3Class' 
DIR_AUDIO = DIR + '/soundfiles/TAU-urban-acoustic-scenes-2020-3class-development/'
df_meta = pd.read_csv(f'{DIR}/meta.csv')
df_meta = pd.read_csv(io.StringIO(df_meta.to_csv(index=False)))
df_meta.head()

,filename,scene_label,identifier
0,audio/airport-barcelona-0-0-a.wav,indoor,barcelona-0
1,audio/airport-barcelona-0-10-a.wav,indoor,barcelona-0
2,audio/airport-barcelona-0-11-a.wav,indoor,barcelona-0
3,audio/airport-barcelona-0-12-a.wav,indoor,barcelona-0
4,audio/airport-barcelona-0-13-a.wav,indoor,barcelona-0


time: 105 ms


In [0]:
df_meta.values.shape

(14400, 3)

time: 3.46 ms


In [0]:
df_meta = df_meta[df_meta['scene_label'] != 'transportation']

time: 5.85 ms


In [0]:
print(df_meta.loc[df_meta['scene_label'] == 'outdoor'].shape)
print(df_meta.loc[df_meta['scene_label'] == 'indoor'].shape)
print(df_meta.loc[df_meta['scene_label'] == 'transportation'].shape)

(5760, 3)
(4320, 3)
(0, 3)
time: 8.14 ms


Reading the datasets and storing it to 'signals'

In [0]:
info_sound = df_meta[['filename','scene_label']].to_numpy()
np.unique(info_sound).shape

(10082,)

time: 18.2 ms


In [0]:
LABELS = ['indoor', 'outdoor']

time: 1.06 ms


In [0]:
FILEPATH = '/content/drive/My Drive/DCASE 2020 Challenge/TAU Urban Acoustic Scenes 2020 3Class/features.csv'
BASE_PATH = '/content/drive/My Drive/DCASE 2020 Challenge/TAU Urban Acoustic Scenes 2020 3Class/'

time: 1.26 ms


Labels:


0.   Indoor
1.   Outdoor
2.   Transportation



# Getting Features
1.   5 MFCCs mean and std
2.   Centroid Mean
3.   Centroid STD
4.   Flatness Mean
5.   Flatness STD
6.   RMS Mean
7.   RMS Std



In [0]:
PARTS = 10
def get_features(filename, signal, label)->pd.DataFrame:
    def get_centroid(y, sr):
        return librosa.feature.spectral_centroid(y=y, sr=sr)
    def get_flatness(y):
        return librosa.feature.spectral_flatness(y=y)
    def get_rms(y):
        return librosa.feature.rms(y=y)
    def get_mfccs(y, sr):
        return librosa.feature.mfcc(y=y, sr=sr, n_mfcc=5)

    feats = (np.zeros((1,18))[0])
    
    music_features = []
    signal = signal[:,0]
    for second in np.array_split(signal, PARTS):
        arr_mfcc = []
        #print(second.shape)
        c = get_centroid(second, SAMPLERATE)
        f = get_flatness(second)
        rms = get_rms(second)
        #print(r)
        mfccs = get_mfccs(second, SAMPLERATE)
        #print(np.array(mfccs).shape)
        mfccs_mean = [np.mean(x) for x in mfccs]
        #print(mfccs_mean)
        
        #mfccs_mean = np.array(mfccs).reshape(1,-1)
        mfccs_std = [np.std(x) for x in mfccs]
        #mfccs_std = np.array(mfccs).reshape(1,-1)
     
        #print(np.array(mfccs).shape)
        music_features = np.hstack([arr_mfcc, np.array([np.mean(c),np.std(c),np.mean(f), np.std(f),np.mean(rms),np.std(rms)]), mfccs_mean, mfccs_std, filename, label]) 
        feats = np.vstack([feats, music_features])  
            
    return pd.DataFrame(data=feats).drop(0).reset_index(drop=True)

time: 24.9 ms


In [0]:
SAMPLERATE = 48000
for filename, label in info_sound:
    if(path.isfile(os.path.join(DIR_AUDIO, filename))):
        signal, sr = sf.read(os.path.join(DIR_AUDIO, filename))
        signal = signal - np.mean(signal)
        signal = signal / np.std(signal)
        feats = get_features(filename, signal, label)
        #print(feats.iloc[0,:])
        if (create_new): 
            feats.to_csv(FILEPATH, mode='w', header=True, index=False)
            create_new = False
        else:
            feats.to_csv(FILEPATH, mode='a', header=False, index=False)

"SAMPLERATE = 48000\nfor filename, label in info_sound:\n    if(path.isfile(os.path.join(DIR_AUDIO, filename))):\n        signal, sr = sf.read(os.path.join(DIR_AUDIO, filename))\n        signal = signal - np.mean(signal)\n        signal = signal / np.std(signal)\n        feats = get_features(filename, signal, label)\n        #print(feats.iloc[0,:])\n        if (create_new): \n            feats.to_csv(FILEPATH, mode='w', header=True, index=False)\n            create_new = False\n        else:\n            feats.to_csv(FILEPATH, mode='a', header=False, index=False)"

time: 89.1 ms


DF: Descrição

        features
*   0: média centroide 
*   1: std centroide
*   2: media flatness
*   3: std flatness
*   4: média rms
*   5: std rms
*   6-10: media mfccs
*   11-15: std mfccs


        outros
*   16: name
*   17: label




